In [3]:
import torch.nn.functional as F # pytorch 激活函数的类
from torch import nn,optim # 构建模型和优化器
import torch
import pickle as pk
import numpy as np
import jieba

class Grade(nn.Module):
    def __init__(self, parameter):
        super(Grade, self).__init__()
        embedding_dim = parameter['embedding_dim']
        hidden_size = parameter['hidden_size']
        num_layers = parameter['num_layers']
        dropout = parameter['dropout']
        word_size = parameter['word_size']
        self.embedding = nn.Embedding(word_size, embedding_dim, padding_idx=0)
        
        self.lstm_q = nn.LSTM(embedding_dim, hidden_size, num_layers, bidirectional=True, batch_first=True, dropout=dropout)

        self.lstm_a = nn.LSTM(embedding_dim, hidden_size, num_layers, bidirectional=True, batch_first=True, dropout=dropout)

        self.dropout = nn.Dropout(p=dropout)

        
    def forward(self, q, a1,a2 = None):
        q_emd = self.embedding(q)
        q_emd,(h, c)= self.lstm_q(q_emd)
        q_emd = torch.max(q_emd,1)[0]

        a1_emd = self.embedding(a1)
        a1_emd,(h, c)= self.lstm_a(a1_emd)
        a1_emd = torch.max(a1_emd,1)[0]
        if a2 is not None:
            a2_emd = self.embedding(a2)
            a2_emd,(h, c)= self.lstm_a(a2_emd)
            a2_emd = torch.max(a2_emd,1)[0]
            return q_emd,a1_emd,a2_emd
        return F.cosine_similarity(q_emd,a1_emd,1,1e-8)


    
def list2torch(a):
    return torch.from_numpy(np.array(a)).long().to(parameter['cuda'])

def predict(model,parameter,q,a):
    q = list(q)
    a = list(a)
    q_cut = []
    for i in q:
        if i in parameter['word2id']:
            q_cut.append(parameter['word2id'][i])
        else:#unk 替换不识别的字
            q_cut.append(parameter['word2id']['<UNK>'])
    a_cut = []
    for i in a:
        if i in parameter['word2id']:
            a_cut.append(parameter['word2id'][i])
        else:
            a_cut.append(parameter['word2id']['<UNK>'])
    print(q_cut,a_cut)
    q_cut,a_cut = [q_cut[:parameter['max_len']]],[a_cut[:parameter['max_len']]]
    prob = model(list2torch(q_cut),list2torch(a_cut))#制作tensor
    return prob.cpu().item()

def load_model(root_path = './'):
    parameter = pk.load(open(root_path+'parameter.pkl','rb'))
#     print(parameter)
    model = Grade(parameter).to(parameter['cuda'])
    model.load_state_dict(torch.load(root_path+'grade.h5'))
    model.eval()
    return model,parameter

In [21]:
root_path = './'
parameter = pk.load(open(root_path+'parameter.pkl','rb'))
parameter

{'epoch': 100,
 'batch_size': 300,
 'embedding_dim': 300,
 'hidden_size': 128,
 'num_layers': 2,
 'dropout': 0.1,
 'cuda': device(type='cpu'),
 'lr': 0.001,
 'max_len': 50,
 'qa_list': [[['A', 'u', 't', 'o', 'M', 'L', '问', '题', '构', '成', '?'],
   ['特', '征', '选', '择'],
   ['模', '型', '选', '择'],
   ['算', '法', '选', '择']],
  [['特', '征', '工', '程', '选', '择', '思', '路', '？'],
   ['有', '监', '督', '的', '特', '征', '选', '择'],
   ['基',
    '于',
    '模',
    '型',
    '，',
    'l',
    'r',
    '的',
    '系',
    '数',
    '，',
    '树',
    '模',
    '型',
    '的',
    'i',
    'm',
    'p',
    'o',
    'r',
    't',
    'a',
    'n',
    'c',
    'e',
    '等',
    '等'],
   ['基', '于', '选', '择', '，', '前', '项', '后', '项', '选', '择'],
   ['无', '监', '督', '的', '特', '征', '选', '择'],
   ['基',
    '于',
    '统',
    '计',
    '信',
    '息',
    '的',
    '，',
    '熵',
    '、',
    '相',
    '关',
    '性',
    '、',
    'K',
    'L',
    '系',
    '数'],
   ['基',
    '于',
    '方',
    '差',
    '，',
    '因',
    '子',
    '分',
 

In [4]:
model,parameter = load_model()
q = '讲讲贝叶斯优化如何在automl上应用？'
a = '目的：通过拟合参数和模型能力之间的关系：模型能力=f(超参数)，找到最合适的超参数'
prob = predict(model,parameter,q,a)
prob

[120, 120, 111, 112, 113, 72, 94, 121, 122, 123, 41, 3, 4, 5, 39, 33, 124, 125, 104, 25] [126, 29, 127, 128, 129, 109, 110, 85, 36, 130, 17, 18, 131, 106, 132, 133, 29, 57, 35, 127, 17, 18, 131, 106, 134, 135, 136, 137, 85, 36, 138, 32, 139, 140, 141, 110, 142, 29, 137, 85, 36]


0.6445427536964417